# Document Binarization Model Training Notebook

This notebook will guide you through training the **EfficientNet-B0 + InceptionNet-V3** model for document binarization.

## 📋 Overview
- **Architecture**: Dual encoder (EfficientNet-B0 + InceptionNet-V3) with UNet decoder
- **Dataset**: DIBCO cleaned and preprocessed patches (256×256)
- **Task**: Document image binarization (segmentation)

## 🗂️ Data Structure
```
split/
  ├── train/
  │   ├── images/  (training image patches .npy)
  │   └── gt/      (training ground truth .npy)
  ├── val/
  │   ├── images/  (validation patches)
  │   └── gt/
  └── test/
      ├── images/  (test patches)
      └── gt/
```

## Step 1: Import Required Libraries

In [ ]:
# Standard libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Import your model
from model import DocumentBinarizationModel, CompleteBinarizationPipeline

# Check PyTorch and CUDA
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    device = torch.device('cuda')
else:
    print("Using CPU")
    device = torch.device('cpu')

## Step 2: Configure Paths and Hyperparameters

In [ ]:
# Paths
PROJECT_ROOT = Path("/Users/shreyatiwari/Documents/Soft Computing Project")
TRAIN_IMG_DIR = PROJECT_ROOT / "split/train/images"
TRAIN_GT_DIR = PROJECT_ROOT / "split/train/gt"
VAL_IMG_DIR = PROJECT_ROOT / "split/val/images"
VAL_GT_DIR = PROJECT_ROOT / "split/val/gt"
TEST_IMG_DIR = PROJECT_ROOT / "split/test/images"
TEST_GT_DIR = PROJECT_ROOT / "split/test/gt"
CHECKPOINT_DIR = PROJECT_ROOT / "checkpoints"
CHECKPOINT_DIR.mkdir(exist_ok=True)

# Training Configuration
CONFIG = {
    # Data
    'batch_size': 8,  # Adjust based on GPU memory
    'num_workers': 4,  # For data loading
    'input_size': 256,  # Patch size (already 256x256)
    
    # Training
    'num_epochs': 50,
    'learning_rate': 1e-4,
    'weight_decay': 1e-5,
    'grad_clip': 1.0,
    
    # Loss weights
    'bce_weight': 1.0,
    'dice_weight': 1.0,
    'edge_weight': 0.5,
    
    # Scheduler
    'scheduler_patience': 5,
    'scheduler_factor': 0.5,
    
    # Checkpointing
    'save_every': 5,  # Save checkpoint every N epochs
    'best_metric': 'val_f1',  # Metric to track for best model
}

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## Step 3: Verify Dataset

In [ ]:
# Count files in each split
train_images = sorted(list(TRAIN_IMG_DIR.glob("*.npy")))
train_gts = sorted(list(TRAIN_GT_DIR.glob("*.npy")))
val_images = sorted(list(VAL_IMG_DIR.glob("*.npy")))
val_gts = sorted(list(VAL_GT_DIR.glob("*.npy")))
test_images = sorted(list(TEST_IMG_DIR.glob("*.npy")))
test_gts = sorted(list(TEST_GT_DIR.glob("*.npy")))

print("Dataset Statistics:")
print(f"  Training:   {len(train_images)} images, {len(train_gts)} GTs")
print(f"  Validation: {len(val_images)} images, {len(val_gts)} GTs")
print(f"  Test:       {len(test_images)} images, {len(test_gts)} GTs")
print(f"  Total:      {len(train_images) + len(val_images) + len(test_images)} patches")

# Verify matching counts
assert len(train_images) == len(train_gts), "Mismatch in training data!"
assert len(val_images) == len(val_gts), "Mismatch in validation data!"
assert len(test_images) == len(test_gts), "Mismatch in test data!"
print("\n✅ All image-GT pairs matched!")

## Step 4: Explore Sample Data

In [ ]:
# Load and visualize random samples
np.random.seed(42)
sample_idx = np.random.randint(0, len(train_images))

sample_img = np.load(train_images[sample_idx])
sample_gt = np.load(train_gts[sample_idx])

print(f"Sample: {train_images[sample_idx].name}")
print(f"Image shape: {sample_img.shape}, dtype: {sample_img.dtype}")
print(f"Image range: [{sample_img.min():.3f}, {sample_img.max():.3f}]")
print(f"GT shape: {sample_gt.shape}, dtype: {sample_gt.dtype}")
print(f"GT range: [{sample_gt.min():.3f}, {sample_gt.max():.3f}]")
print(f"GT unique values: {np.unique(sample_gt)}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(sample_img, cmap='gray')
axes[0].set_title('Input Image')
axes[0].axis('off')

axes[1].imshow(sample_gt, cmap='gray')
axes[1].set_title('Ground Truth')
axes[1].axis('off')

# Overlay
axes[2].imshow(sample_img, cmap='gray', alpha=0.7)
axes[2].imshow(sample_gt, cmap='Reds', alpha=0.3)
axes[2].set_title('Overlay')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## Step 5: Create PyTorch Dataset Class

In [ ]:
class BinarizationDataset(Dataset):
    """Dataset for document binarization with .npy patches"""
    
    def __init__(self, image_dir, gt_dir, augment=False):
        self.image_dir = Path(image_dir)
        self.gt_dir = Path(gt_dir)
        self.augment = augment
        
        # Get all image files
        self.image_files = sorted(list(self.image_dir.glob("*.npy")))
        self.gt_files = sorted(list(self.gt_dir.glob("*.npy")))
        
        print(f"Loaded {len(self.image_files)} samples from {image_dir.name}")
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load data
        image = np.load(self.image_files[idx]).astype(np.float32)
        gt = np.load(self.gt_files[idx]).astype(np.float32)
        
        # Ensure image is in [0, 1] range
        if image.max() > 1.0:
            image = image / 255.0
            
        # Ensure GT is binary {0, 1}
        if gt.max() > 1.0:
            gt = (gt > 127).astype(np.float32)
        
        # Add channel dimension if needed
        if len(image.shape) == 2:
            image = image[np.newaxis, :, :]  # (1, H, W)
        
        if len(gt.shape) == 2:
            gt = gt[np.newaxis, :, :]  # (1, H, W)
        
        # Convert to 3-channel for pretrained models
        if image.shape[0] == 1:
            image = np.repeat(image, 3, axis=0)  # (3, H, W)
        
        # Data augmentation
        if self.augment:
            # Random horizontal flip
            if np.random.rand() > 0.5:
                image = np.flip(image, axis=2).copy()
                gt = np.flip(gt, axis=2).copy()
            
            # Random vertical flip
            if np.random.rand() > 0.5:
                image = np.flip(image, axis=1).copy()
                gt = np.flip(gt, axis=1).copy()
            
            # Random 90-degree rotation
            if np.random.rand() > 0.5:
                k = np.random.randint(1, 4)
                image = np.rot90(image, k, axes=(1, 2)).copy()
                gt = np.rot90(gt, k, axes=(1, 2)).copy()
        
        # Convert to torch tensors
        image = torch.from_numpy(image)
        gt = torch.from_numpy(gt)
        
        return image, gt

# Create datasets
train_dataset = BinarizationDataset(TRAIN_IMG_DIR, TRAIN_GT_DIR, augment=True)
val_dataset = BinarizationDataset(VAL_IMG_DIR, VAL_GT_DIR, augment=False)
test_dataset = BinarizationDataset(TEST_IMG_DIR, TEST_GT_DIR, augment=False)

print(f"\n✅ Datasets created successfully!")

## Step 6: Create DataLoaders

In [ ]:
# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if torch.cuda.is_available() else False
)

print("DataLoader Statistics:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")
print(f"  Batch size: {CONFIG['batch_size']}")

# Test dataloader
sample_batch = next(iter(train_loader))
print(f"\nSample batch:")
print(f"  Images shape: {sample_batch[0].shape}")
print(f"  GTs shape: {sample_batch[1].shape}")
print(f"✅ DataLoaders working correctly!")

## Step 7: Initialize Model

In [ ]:
# Initialize the complete binarization pipeline
model = CompleteBinarizationPipeline(in_channels=3, out_channels=1)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Model Information:")
print(f"  Architecture: EfficientNet-B0 + InceptionNet-V3 + UNet Decoder")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Device: {device}")

# Test forward pass
with torch.no_grad():
    test_input = torch.randn(2, 3, 256, 256).to(device)
    test_output = model(test_input)
    print(f"\nTest forward pass:")
    print(f"  Input shape: {test_input.shape}")
    print(f"  Output shape: {test_output.shape}")
    print(f"✅ Model initialized successfully!")

## Step 8: Define Loss Function and Metrics

In [ ]:
class CombinedLoss(nn.Module):
    """Combined BCE + Dice + Edge Loss"""
    
    def __init__(self, bce_weight=1.0, dice_weight=1.0, edge_weight=0.5):
        super().__init__()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
        self.edge_weight = edge_weight
        self.bce = nn.BCEWithLogitsLoss()
        
    def dice_loss(self, pred, target, smooth=1e-6):
        pred = torch.sigmoid(pred)
        pred_flat = pred.view(-1)
        target_flat = target.view(-1)
        intersection = (pred_flat * target_flat).sum()
        dice = (2. * intersection + smooth) / (pred_flat.sum() + target_flat.sum() + smooth)
        return 1 - dice
    
    def edge_loss(self, pred, target):
        """Edge-aware loss using Sobel filters"""
        # Sobel kernels
        sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)
        sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)
        
        pred_sigmoid = torch.sigmoid(pred)
        
        # Compute edges
        pred_edge_x = torch.nn.functional.conv2d(pred_sigmoid, sobel_x, padding=1)
        pred_edge_y = torch.nn.functional.conv2d(pred_sigmoid, sobel_y, padding=1)
        pred_edge = torch.sqrt(pred_edge_x**2 + pred_edge_y**2)
        
        target_edge_x = torch.nn.functional.conv2d(target, sobel_x, padding=1)
        target_edge_y = torch.nn.functional.conv2d(target, sobel_y, padding=1)
        target_edge = torch.sqrt(target_edge_x**2 + target_edge_y**2)
        
        return torch.nn.functional.mse_loss(pred_edge, target_edge)
    
    def forward(self, pred, target):
        bce = self.bce(pred, target)
        dice = self.dice_loss(pred, target)
        edge = self.edge_loss(pred, target)
        
        total = (self.bce_weight * bce + 
                 self.dice_weight * dice + 
                 self.edge_weight * edge)
        
        return total, {'bce': bce.item(), 'dice': dice.item(), 'edge': edge.item()}

def calculate_metrics(pred, target, threshold=0.5):
    """Calculate accuracy, precision, recall, F1, IoU"""
    pred_binary = (torch.sigmoid(pred) > threshold).float()
    target_binary = target
    
    # True Positives, False Positives, False Negatives, True Negatives
    tp = ((pred_binary == 1) & (target_binary == 1)).float().sum()
    fp = ((pred_binary == 1) & (target_binary == 0)).float().sum()
    fn = ((pred_binary == 0) & (target_binary == 1)).float().sum()
    tn = ((pred_binary == 0) & (target_binary == 0)).float().sum()
    
    # Metrics
    accuracy = (tp + tn) / (tp + fp + fn + tn + 1e-8)
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    iou = tp / (tp + fp + fn + 1e-8)
    
    return {
        'accuracy': accuracy.item(),
        'precision': precision.item(),
        'recall': recall.item(),
        'f1': f1.item(),
        'iou': iou.item()
    }

# Initialize loss
criterion = CombinedLoss(
    bce_weight=CONFIG['bce_weight'],
    dice_weight=CONFIG['dice_weight'],
    edge_weight=CONFIG['edge_weight']
)

print("✅ Loss function and metrics defined!")

## Step 9: Configure Optimizer and Scheduler

In [ ]:
# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',  # Maximize F1 score
    factor=CONFIG['scheduler_factor'],
    patience=CONFIG['scheduler_patience'],
    verbose=True
)

print("Optimizer Configuration:")
print(f"  Type: AdamW")
print(f"  Learning rate: {CONFIG['learning_rate']}")
print(f"  Weight decay: {CONFIG['weight_decay']}")
print(f"  Scheduler: ReduceLROnPlateau (patience={CONFIG['scheduler_patience']})")
print(f"✅ Optimizer and scheduler configured!")

## Step 10: Training Loop Functions

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch):
    """Train for one epoch"""
    model.train()
    
    running_loss = 0.0
    running_metrics = {
        'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 
        'f1': 0.0, 'iou': 0.0,
        'bce': 0.0, 'dice': 0.0, 'edge': 0.0
    }
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch} [Train]')
    
    for batch_idx, (images, targets) in enumerate(pbar):
        images = images.to(device)
        targets = targets.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        
        # Compute loss
        loss, loss_components = criterion(outputs, targets)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG['grad_clip'])
        
        optimizer.step()
        
        # Calculate metrics
        with torch.no_grad():
            metrics = calculate_metrics(outputs, targets)
        
        # Update running metrics
        running_loss += loss.item()
        for key in running_metrics:
            if key in metrics:
                running_metrics[key] += metrics[key]
            elif key in loss_components:
                running_metrics[key] += loss_components[key]
        
        # Update progress bar
        pbar.set_postfix({
            'loss': loss.item(),
            'f1': metrics['f1'],
            'iou': metrics['iou']
        })
    
    # Average metrics
    num_batches = len(dataloader)
    avg_loss = running_loss / num_batches
    avg_metrics = {k: v / num_batches for k, v in running_metrics.items()}
    
    return avg_loss, avg_metrics


def validate(model, dataloader, criterion, device, epoch):
    """Validate the model"""
    model.eval()
    
    running_loss = 0.0
    running_metrics = {
        'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0,
        'f1': 0.0, 'iou': 0.0,
        'bce': 0.0, 'dice': 0.0, 'edge': 0.0
    }
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch} [Val]')
    
    with torch.no_grad():
        for images, targets in pbar:
            images = images.to(device)
            targets = targets.to(device)
            
            # Forward pass
            outputs = model(images)
            
            # Compute loss
            loss, loss_components = criterion(outputs, targets)
            
            # Calculate metrics
            metrics = calculate_metrics(outputs, targets)
            
            # Update running metrics
            running_loss += loss.item()
            for key in running_metrics:
                if key in metrics:
                    running_metrics[key] += metrics[key]
                elif key in loss_components:
                    running_metrics[key] += loss_components[key]
            
            # Update progress bar
            pbar.set_postfix({
                'loss': loss.item(),
                'f1': metrics['f1'],
                'iou': metrics['iou']
            })
    
    # Average metrics
    num_batches = len(dataloader)
    avg_loss = running_loss / num_batches
    avg_metrics = {k: v / num_batches for k, v in running_metrics.items()}
    
    return avg_loss, avg_metrics

print("✅ Training functions defined!")

## Step 11: Train the Model 🚀

**This is the main training loop. Run this cell to start training!**

Training will:
- Save checkpoints every 5 epochs
- Save the best model based on validation F1 score
- Track all metrics and losses
- Automatically adjust learning rate based on validation performance

In [ ]:
# Training history
history = {
    'train_loss': [], 'val_loss': [],
    'train_f1': [], 'val_f1': [],
    'train_iou': [], 'val_iou': [],
    'train_accuracy': [], 'val_accuracy': [],
    'train_precision': [], 'val_precision': [],
    'train_recall': [], 'val_recall': [],
    'learning_rates': []
}

# Best model tracking
best_metric = 0.0
best_epoch = 0

print("=" * 80)
print("STARTING TRAINING")
print("=" * 80)

for epoch in range(1, CONFIG['num_epochs'] + 1):
    print(f"\n{'='*80}")
    print(f"Epoch {epoch}/{CONFIG['num_epochs']}")
    print(f"{'='*80}")
    
    # Train
    train_loss, train_metrics = train_one_epoch(
        model, train_loader, criterion, optimizer, device, epoch
    )
    
    # Validate
    val_loss, val_metrics = validate(
        model, val_loader, criterion, device, epoch
    )
    
    # Learning rate scheduling
    current_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_metrics['f1'])
    
    # Save history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_f1'].append(train_metrics['f1'])
    history['val_f1'].append(val_metrics['f1'])
    history['train_iou'].append(train_metrics['iou'])
    history['val_iou'].append(val_metrics['iou'])
    history['train_accuracy'].append(train_metrics['accuracy'])
    history['val_accuracy'].append(val_metrics['accuracy'])
    history['train_precision'].append(train_metrics['precision'])
    history['val_precision'].append(val_metrics['precision'])
    history['train_recall'].append(train_metrics['recall'])
    history['val_recall'].append(val_metrics['recall'])
    history['learning_rates'].append(current_lr)
    
    # Print epoch summary
    print(f"\nEpoch {epoch} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    print(f"  Train F1: {train_metrics['f1']:.4f} | Val F1: {val_metrics['f1']:.4f}")
    print(f"  Train IoU: {train_metrics['iou']:.4f} | Val IoU: {val_metrics['iou']:.4f}")
    print(f"  Learning Rate: {current_lr:.6f}")
    
    # Save checkpoint periodically
    if epoch % CONFIG['save_every'] == 0:
        checkpoint_path = CHECKPOINT_DIR / f"checkpoint_epoch_{epoch}.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_f1': val_metrics['f1'],
            'history': history
        }, checkpoint_path)
        print(f"  💾 Checkpoint saved: {checkpoint_path.name}")
    
    # Save best model
    if val_metrics['f1'] > best_metric:
        best_metric = val_metrics['f1']
        best_epoch = epoch
        best_model_path = CHECKPOINT_DIR / "best_model.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_f1': val_metrics['f1'],
            'val_iou': val_metrics['iou'],
            'history': history
        }, best_model_path)
        print(f"  ⭐ New best model saved! F1: {best_metric:.4f}")

print("\n" + "=" * 80)
print("TRAINING COMPLETED!")
print("=" * 80)
print(f"Best validation F1: {best_metric:.4f} (Epoch {best_epoch})")
print(f"Checkpoints saved in: {CHECKPOINT_DIR}")

## Step 12: Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

epochs = range(1, len(history['train_loss']) + 1)

# Loss
axes[0, 0].plot(epochs, history['train_loss'], 'b-', label='Train Loss')
axes[0, 0].plot(epochs, history['val_loss'], 'r-', label='Val Loss')
axes[0, 0].set_title('Loss over Epochs')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# F1 Score
axes[0, 1].plot(epochs, history['train_f1'], 'b-', label='Train F1')
axes[0, 1].plot(epochs, history['val_f1'], 'r-', label='Val F1')
axes[0, 1].set_title('F1 Score over Epochs')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('F1 Score')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# IoU
axes[1, 0].plot(epochs, history['train_iou'], 'b-', label='Train IoU')
axes[1, 0].plot(epochs, history['val_iou'], 'r-', label='Val IoU')
axes[1, 0].set_title('IoU over Epochs')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('IoU')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Learning Rate
axes[1, 1].plot(epochs, history['learning_rates'], 'g-')
axes[1, 1].set_title('Learning Rate over Epochs')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Learning Rate')
axes[1, 1].set_yscale('log')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(CHECKPOINT_DIR / 'training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Training history plot saved to {CHECKPOINT_DIR / 'training_history.png'}")

## Step 13: Evaluate on Test Set

In [ ]:
# Load best model
best_checkpoint = torch.load(CHECKPOINT_DIR / "best_model.pth")
model.load_state_dict(best_checkpoint['model_state_dict'])
print(f"Loaded best model from epoch {best_checkpoint['epoch']}")
print(f"Validation F1: {best_checkpoint['val_f1']:.4f}")

# Evaluate on test set
model.eval()
test_loss = 0.0
test_metrics = {
    'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0,
    'f1': 0.0, 'iou': 0.0
}

print("\nEvaluating on test set...")
with torch.no_grad():
    for images, targets in tqdm(test_loader, desc='Testing'):
        images = images.to(device)
        targets = targets.to(device)
        
        # Forward pass
        outputs = model(images)
        
        # Compute loss
        loss, _ = criterion(outputs, targets)
        test_loss += loss.item()
        
        # Calculate metrics
        metrics = calculate_metrics(outputs, targets)
        for key in test_metrics:
            test_metrics[key] += metrics[key]

# Average metrics
num_batches = len(test_loader)
test_loss /= num_batches
for key in test_metrics:
    test_metrics[key] /= num_batches

print("\n" + "=" * 80)
print("TEST SET RESULTS")
print("=" * 80)
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_metrics['accuracy']:.4f}")
print(f"  Precision: {test_metrics['precision']:.4f}")
print(f"  Recall: {test_metrics['recall']:.4f}")
print(f"  F1 Score: {test_metrics['f1']:.4f}")
print(f"  IoU: {test_metrics['iou']:.4f}")
print("=" * 80)

## Step 14: Visualize Predictions on Test Samples

In [ ]:
# Visualize predictions on random test samples
model.eval()
num_samples = 4
sample_indices = np.random.choice(len(test_dataset), num_samples, replace=False)

fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4*num_samples))

with torch.no_grad():
    for idx, sample_idx in enumerate(sample_indices):
        image, gt = test_dataset[sample_idx]
        image_batch = image.unsqueeze(0).to(device)
        
        # Predict
        output = model(image_batch)
        pred = torch.sigmoid(output).cpu().squeeze().numpy()
        pred_binary = (pred > 0.5).astype(np.float32)
        
        # Convert to numpy for visualization
        image_np = image[0].cpu().numpy()  # First channel for grayscale
        gt_np = gt.squeeze().cpu().numpy()
        
        # Plot
        axes[idx, 0].imshow(image_np, cmap='gray')
        axes[idx, 0].set_title('Input Image')
        axes[idx, 0].axis('off')
        
        axes[idx, 1].imshow(gt_np, cmap='gray')
        axes[idx, 1].set_title('Ground Truth')
        axes[idx, 1].axis('off')
        
        axes[idx, 2].imshow(pred, cmap='gray')
        axes[idx, 2].set_title('Prediction (Probability)')
        axes[idx, 2].axis('off')
        
        axes[idx, 3].imshow(pred_binary, cmap='gray')
        axes[idx, 3].set_title('Prediction (Binary)')
        axes[idx, 3].axis('off')

plt.tight_layout()
plt.savefig(CHECKPOINT_DIR / 'test_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Test predictions saved to {CHECKPOINT_DIR / 'test_predictions.png'}")

## 🎉 Training Complete!

### Summary
- ✅ Model trained successfully with EfficientNet-B0 + InceptionNet-V3
- ✅ Best model saved based on validation F1 score
- ✅ Training history and predictions visualized
- ✅ All checkpoints saved in `checkpoints/` directory

### Next Steps
1. **Test on new images**: Use `inference.py` to test on unseen documents
2. **Fine-tune hyperparameters**: Adjust learning rate, batch size, loss weights
3. **Analyze errors**: Look at difficult samples where the model struggles
4. **Deploy**: Use the best model for production inference

### Files Generated
- `checkpoints/best_model.pth` - Best model checkpoint
- `checkpoints/checkpoint_epoch_*.pth` - Periodic checkpoints
- `checkpoints/training_history.png` - Training curves
- `checkpoints/test_predictions.png` - Sample predictions